In [1]:
import pandas as pd
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import joblib
df = pd.read_csv("miss_smaler_7per.csv")


In [2]:
df = df[['home_batting_batting_avg_10RA',
         'away_batting_batting_avg_10RA',
        'date',
        "home_pitching_earned_run_avg_10RA",
        'away_pitching_earned_run_avg_10RA',
        'home_batting_onbase_plus_slugging_10RA',
        'away_batting_onbase_plus_slugging_10RA',
        'home_batting_RBI_10RA',
        'away_batting_RBI_10RA',
        'home_team_wins_mean' ,
        'away_team_wins_mean' ,
        "home_pitching_H_batters_faced_10RA" ,
        "away_pitching_H_batters_faced_10RA" ,
        #'home_team_errors_mean',
        #'away_team_errors_mean',
        'home_team_abbr' , 
        'away_team_abbr',
        'home_team_win' ,
        'home_pitching_SO_batters_faced_mean',
        'away_pitching_SO_batters_faced_mean',
        'home_pitching_wpa_def_mean',
         'away_pitching_wpa_def_mean',
        'home_batting_wpa_bat_mean',
        'away_batting_wpa_bat_mean' ,




        ]]
 
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year


df.drop(columns=['date'], inplace=True, errors='ignore')
df

home_batting_batting_avg_10RA  away_batting_batting_avg_10RA  \
0                                NaN                            NaN   
1                                NaN                            NaN   
2                                NaN                            NaN   
3                                NaN                            NaN   
4                                NaN                            NaN   
...                              ...                            ...   
11062                      -0.812161                      -0.478545   
11063                       0.280776                       0.478694   
11064                      -1.639621                      -0.089452   
11065                       0.973774                      -1.697474   
11066                      -0.070894                       0.564776   

       home_pitching_earned_run_avg_10RA  away_pitching_earned_run_avg_10RA  \
0                                    NaN                                NaN   
1                                    NaN                                NaN   
2                                    NaN                                NaN   
3                                    NaN                                NaN   
4                                    NaN                                NaN   
...                                  ...                                ...   
11062                           0.365706                           0.566667   
11063                          -0.222728                          -1.046506   
11064                           0.168184                                NaN   
11065                          -1.077280                          -0.381872   
11066                           0.183886                          -0.818397   

       home_batting_onbase_plus_slugging_10RA  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3                                         NaN   
4                                         NaN   
...                                       ...   
11062                               -0.296017   
11063                                0.154042   
11064                               -1.369143   
11065                                1.353418   
11066                                0.498892   

       away_batting_onbase_plus_slugging_10RA  home_batting_RBI_10RA  \
0                                         NaN                    NaN   
1                                         NaN                    NaN   
2                                         NaN                    NaN   
3                                         NaN                    NaN   
4                                         NaN                    NaN   
...                                       ...                    ...   
11062                               -0.333902              -0.472473   
11063                                0.560315              -0.294963   
11064                                0.093464              -1.537530   
11065                               -1.586968               0.503830   
11066                                0.134110               0.947603   

       away_batting_RBI_10RA  home_team_wins_mean  away_team_wins_mean  ...  \
0                        NaN                  NaN                  NaN  ...   
1                        NaN                  NaN                  NaN  ...   
2                        NaN                  NaN                  NaN  ...   
3                        NaN                  NaN                  NaN  ...   
4                        NaN                  NaN                  NaN  ...   
...                      ...                  ...                  ...  ...   
11062              -0.647647             0.199443             0.791356  ...   
11063               0.327235            -0.966669            -0.093481  ...   
11064              -0.736273            

In [3]:
df['home_bera_10RA'] = df['home_batting_batting_avg_10RA'] - df['home_pitching_earned_run_avg_10RA']
df['away_bera_10RA'] = df['away_batting_batting_avg_10RA'] - df['away_pitching_earned_run_avg_10RA'] 
df['diff_bera_10RA'] = df['home_bera_10RA'] - df['away_bera_10RA']
df['diff_batting_onbase_plus_slugging_10RA'] = df['home_batting_onbase_plus_slugging_10RA'] - df['away_batting_onbase_plus_slugging_10RA']
df['diff_batting_RBI_10RA'] = df['home_batting_RBI_10RA'] - df['away_batting_RBI_10RA']

df['diff_home_pitching_HSO_batters_faced_mean'] = df['home_pitching_SO_batters_faced_mean'] - df['home_pitching_H_batters_faced_10RA']
df['diff_away_pitching_HSO_batters_faced_mean'] = df['away_pitching_SO_batters_faced_mean'] - df['away_pitching_H_batters_faced_10RA']
df['diff_pitching_H_SO_batters_faced_mean']   =  df['diff_home_pitching_HSO_batters_faced_mean'] - df['diff_away_pitching_HSO_batters_faced_mean']
df ['plus_home_pitching_wpa_def_batted_mean']  = df['home_batting_wpa_bat_mean'] +  df['home_pitching_wpa_def_mean']
df ['plus_away_pitching_wpa_def_batted_mean']  = df['away_batting_wpa_bat_mean'] + df['away_pitching_wpa_def_mean']
df['diff_plus_pitching_wpa_def_batted_mean'] = df['plus_home_pitching_wpa_def_batted_mean']-df['plus_away_pitching_wpa_def_batted_mean']



columns_to_drop = [
    'home_batting_batting_avg_10RA', 'away_batting_batting_avg_10RA', 
    'home_batting_onbase_perc_10RA', 'away_batting_onbase_perc_10RA',
    'home_batting_onbase_plus_slugging_10RA','away_batting_onbase_plus_slugging_10RA',
    'home_batting_RBI_10RA','away_batting_RBI_10RA',   
    'home_pitching_earned_run_avg_10RA','away_pitching_earned_run_avg_10RA',
    'home_pitching_wpa_def_mean' , 'away_pitching_wpa_def_mean',
    'home_batting_wpa_bat_mean' , 'away_batting_wpa_bat_mean',
    "home_pitching_SO_batters_faced_mean","away_pitching_SO_batters_faced_mean",
    "plus_home_pitching_wpa_def_batted_mean","plus_away_pitching_wpa_def_batted_mean",
    "home_bera_10RA","away_bera_10RA" ,
    'diff_home_pitching_HSO_batters_faced_mean','diff_away_pitching_HSO_batters_faced_mean'
 ]

df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df

home_team_wins_mean  away_team_wins_mean  \
0                      NaN                  NaN   
1                      NaN                  NaN   
2                      NaN                  NaN   
3                      NaN                  NaN   
4                      NaN                  NaN   
...                    ...                  ...   
11062             0.199443             0.791356   
11063            -0.966669            -0.093481   
11064             0.403748             0.233788   
11065             0.486256             0.938194   
11066             0.486256            -0.011664   

       home_pitching_H_batters_faced_10RA  away_pitching_H_batters_faced_10RA  \
0                                     NaN                                 NaN   
1                                     NaN                                 NaN   
2                                     NaN                                 NaN   
3                                     NaN                                 NaN   
4                                     NaN                                 NaN   
...                                   ...                                 ...   
11062                           -0.546741                           -0.317487   
11063                            0.035824                           -0.897838   
11064                            1.187142                            1.418582   
11065                           -0.611861                           -0.533095   
11066                           -0.020618                           -1.013679   

      home_team_abbr away_team_abbr  home_team_win  year  diff_bera_10RA  \
0                KFH            SAJ          False  2016             NaN   
1                JBM            KJP          False  2016             NaN   
2                VQC            BPH           True  2016             NaN   
3                VJV            MOO          False  2016             NaN   
4                PDF            HAN          False  2016             NaN   
...              ...            ...            ...   ...             ...   
11062            UPV            KFH          False  2023       -0.132654   
11063            GKO            YHA           True  2023       -1.021694   
11064            QPO            XFB          False  2023             NaN   
11065            SAJ            STC          False  2023        3.366657   
11066            HXK            FBW           True  2023       -1.637953   

       diff_batting_onbase_plus_slugging_10RA  diff_batting_RBI_10RA  \
0                                         NaN                    NaN   
1                                         NaN                    NaN   
2                                         NaN                    NaN   
3                                         NaN                    NaN   
4                                         NaN                    NaN   
...                                       ...                    ...   
11062                                0.037885               0.175174   
11063                               -0.406274              -0.622198   
11064                               -1.462607              -0.801257   
11065                                2.940386               1.062851   
11066                                0.364781               1.240748   

       diff_pitching_H_SO_batters_faced_mean  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        NaN   
4                                        NaN   
...                                      ...   
11062                              -0.244036   
11063                              -2.981052   
11064                               0.593788   
11065                               0.707867   
11066                               0.267827   

       diff_plus_pitching_wpa_def_batted_mean  
0                 

In [4]:
# 分離目標變量
y = df['home_team_win'].astype(bool)
X = df.drop(columns=['home_team_win'])

# 編碼分類特徵（隊伍和日期相關）
cat_features = ['home_team_abbr', 'away_team_abbr']
num_features = [col for col in X.columns if col not in cat_features + ['year']]

# 定義預處理器
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
        ('num', 'passthrough', num_features)
    ]
)

pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('imputer', SimpleImputer(strategy='mean')),
    ('clf', SVC(probability=True, random_state=42))  # 使用 SVM
])
 
param_grid = {
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],  
    'clf__gamma': ['scale', 'auto', 0.0001, 0.001, 0.01, 0.1, 1],  
    'clf__class_weight': [None, 'balanced']  
}
grid = list(ParameterGrid(param_grid))

In [5]:
available_years = sorted(df['year'].unique())
print(f"可用的年份: {available_years}")

models = {}

def split_train_test_per_year(df, year, test_size=0.2):
    df_year = df[df['year'] == year]
    n_total = len(df_year)
    if n_total == 0:
        print(f"警告：{year}年沒有數據。")
        return pd.DataFrame(), pd.DataFrame()
    n_test = max(1, int(n_total * test_size))   
    df_train = df_year.iloc[:-n_test]
    df_test = df_year.iloc[-n_test:]
    return df_train, df_test


可用的年份: [2016, 2017, 2018, 2019, 2021, 2022, 2023]


In [6]:
for year in available_years:
    print(f"\n訓練 {year} 年的模型...")
    train_df, test_df = split_train_test_per_year(df, year, test_size=0.2)
    
    if train_df.empty:
        print(f"警告：{year}年的訓練集為空，跳過該年份的模型訓練。")
        continue
    
    X_train = train_df.drop(columns=['home_team_win', 'year'])
    y_train = train_df['home_team_win']
    
    X_test = test_df.drop(columns=['home_team_win', 'year'])
    y_test = test_df['home_team_win']
    
    best_score = -np.inf
    best_params = None
    
    for params in grid:
        pipeline.set_params(**params)
        
        try:
            pipeline.fit(X_train, y_train)
        except Exception as e:
            print(f"錯誤：{year}年使用參數 {params} 訓練失敗。錯誤信息：{e}")
            continue
        
        try:
            # 在測試集上評估
            y_pred = pipeline.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
        except Exception as e:
            print(f"錯誤：{year}年模型在測試集評估時失敗。錯誤信息：{e}")
            continue
        
        # 更新最佳參數
        if acc > best_score:
            best_score = acc
            best_params = params
    
    print(f"最佳參數: {best_params}")
    print(f"驗證最佳準確率: {best_score:.4f}")
    
    pipeline.set_params(**best_params)
    try:
        pipeline.fit(X_train, y_train)
    except Exception as e:
        print(f"錯誤：{year}年使用最佳參數 {best_params} 訓練失敗。錯誤信息：{e}")
        continue
    
    models[year] = pipeline


print("\n所有年份的模型訓練和評估完成。")

joblib.dump(models, 'trained_models.pkl')
print("模型已保存至 'trained_models.pkl'。")


訓練 2016 年的模型...
最佳參數: {'clf__C': 1, 'clf__class_weight': None, 'clf__gamma': 0.001}
驗證最佳準確率: 0.5783

訓練 2017 年的模型...
最佳參數: {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__gamma': 0.001}
驗證最佳準確率: 0.5828

訓練 2018 年的模型...
最佳參數: {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__gamma': 0.0001}
驗證最佳準確率: 0.5670

訓練 2019 年的模型...
最佳參數: {'clf__C': 100, 'clf__class_weight': 'balanced', 'clf__gamma': 0.001}
驗證最佳準確率: 0.5741

訓練 2021 年的模型...
最佳參數: {'clf__C': 1000, 'clf__class_weight': None, 'clf__gamma': 0.0001}
驗證最佳準確率: 0.5623

訓練 2022 年的模型...
最佳參數: {'clf__C': 10, 'clf__class_weight': None, 'clf__gamma': 0.001}
驗證最佳準確率: 0.6242

訓練 2023 年的模型...
最佳參數: {'clf__C': 0.1, 'clf__class_weight': 'balanced', 'clf__gamma': 0.001}
驗證最佳準確率: 0.5674

所有年份的模型訓練和評估完成。
模型已保存至 'trained_models.pkl'。


In [9]:
#test
test_df1 = pd.read_csv("test_2.csv")

test_df = test_df1[['home_batting_batting_avg_10RA','id',
                    
         'away_batting_batting_avg_10RA',
        "home_pitching_earned_run_avg_10RA",
        'away_pitching_earned_run_avg_10RA',
        'home_batting_onbase_plus_slugging_10RA',
        'away_batting_onbase_plus_slugging_10RA',
        'home_batting_RBI_10RA',
        'away_batting_RBI_10RA',
        'home_team_wins_mean' ,
        'away_team_wins_mean' ,
        #'home_team_errors_mean',
        #'away_team_errors_mean',
        'home_team_abbr' , 
        'away_team_abbr',
        'home_pitching_SO_batters_faced_mean',
        'away_pitching_SO_batters_faced_mean',
        'home_pitching_wpa_def_mean',
        'away_pitching_wpa_def_mean',
        'home_batting_wpa_bat_mean',
        'away_batting_wpa_bat_mean' ,
        "home_pitching_H_batters_faced_10RA" ,
        "away_pitching_H_batters_faced_10RA" ,




        ]]

test_df['year'] = test_df1['season']
test_df['diff_home_pitching_HSO_batters_faced_mean'] = test_df['home_pitching_SO_batters_faced_mean'] - test_df['home_pitching_H_batters_faced_10RA']
test_df['diff_away_pitching_HSO_batters_faced_mean'] = test_df['away_pitching_SO_batters_faced_mean'] - test_df['away_pitching_H_batters_faced_10RA']
test_df['diff_pitching_H_SO_batters_faced_mean']   =  test_df['diff_home_pitching_HSO_batters_faced_mean'] - test_df['diff_away_pitching_HSO_batters_faced_mean']
test_df['home_bera_10RA'] = test_df['home_batting_batting_avg_10RA'] - test_df['home_pitching_earned_run_avg_10RA']
test_df['away_bera_10RA'] = test_df['away_batting_batting_avg_10RA'] - test_df['away_pitching_earned_run_avg_10RA'] 
test_df['diff_bera_10RA'] = test_df['home_bera_10RA'] - test_df['away_bera_10RA']

test_df['diff_batting_onbase_plus_slugging_10RA'] = test_df['home_batting_onbase_plus_slugging_10RA'] - test_df['away_batting_onbase_plus_slugging_10RA']

test_df['diff_batting_RBI_10RA'] = test_df['home_batting_RBI_10RA'] - test_df['away_batting_RBI_10RA']
test_df ['plus_home_pitching_wpa_def_batted_mean']  = test_df['home_batting_wpa_bat_mean'] +  test_df['home_pitching_wpa_def_mean']
test_df ['plus_away_pitching_wpa_def_batted_mean']  = test_df['away_batting_wpa_bat_mean'] + test_df['away_pitching_wpa_def_mean']
test_df['diff_plus_pitching_wpa_def_batted_mean'] = test_df['plus_home_pitching_wpa_def_batted_mean']-test_df['plus_away_pitching_wpa_def_batted_mean']




test_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_df


C:\Users\cheeh\AppData\Local\Temp\ipykernel_3228\2292116997.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['year'] = test_df1['season']
C:\Users\cheeh\AppData\Local\Temp\ipykernel_3228\2292116997.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['diff_home_pitching_HSO_batters_faced_mean'] = test_df['home_pitching_SO_batters_faced_mean'] - test_df['home_pitching_H_batters_faced_10RA']
C:\Users\cheeh\AppData\Local\Temp\ipykernel_3228\2292116997.py:35: SettingWithCopyWarning: 
A value

id  home_team_wins_mean  away_team_wins_mean home_team_abbr  \
0        0             1.183823            -0.248190            HXK   
1        1            -0.457296             0.519884            VJV   
2        2             0.311179            -0.011664            BPH   
3        3            -1.781596            -0.476268            GLO   
4        4                  NaN             0.843798            PDF   
...    ...                  ...                  ...            ...   
6180  6180            -0.008791            -1.124522            PJT   
6181  6181             1.586734            -0.277152            RLJ   
6182  6182             0.080452            -0.100160            VJV   
6183  6183             0.416354             0.556542            KFH   
6184  6184             0.951118             1.177324            UPV   

     away_team_abbr  home_pitching_H_batters_faced_10RA  \
0               SAJ                            1.115384   
1               DPS                           -0.546086   
2               KJP                            0.825120   
3               PJT                            0.930317   
4               HAN                            1.482664   
...             ...                                 ...   
6180            GLO                            0.045047   
6181            KJP                           -0.680187   
6182            DPS                            1.138208   
6183            FBW                           -0.135249   
6184            KJP                           -1.035394   

      away_pitching_H_batters_faced_10RA  year  \
0                                    NaN  2017   
1                               0.647184  2023   
2                               1.213894  2016   
3                               0.810532  2019   
4                              -0.563089  2016   
...                                  ...   ...   
6180                            0.472863  2022   
6181                           -1.173127  2018   
6182                            1.556738  2017   
6183                            0.104392  2018   
6184                            1.079775  2022   

      diff_pitching_H_SO_batters_faced_mean  diff_bera_10RA  \
0                                       NaN       -0.010329   
1                                  1.460674       -0.761318   
2                                 -0.621014        0.954321   
3                                  0.003364        0.504871   
4                                 -4.039115       -1.596654   
...                                     ...             ...   
6180                               1.859140        2.030292   
6181                               0.280290       -0.210705   
6182                               1.843777             NaN   
6183                              -0.543878             NaN   
6184                               1.576253             NaN   

      diff_batting_onbase_plus_slugging_10RA  diff_batting_RBI_10RA  \
0                                  -0.446938              -0.620906   
1                                  -1.352404              -2.040983   
2                                        NaN                    NaN   
3                                   1.202063               0.441697   
4                                   0.912330               0.353588   
...                                      ...                    ...   
6180                                1.917314               0.440534   
6181                               -1.504998                    NaN   
6182                               -0.375928              -0.621552   
6183                                1.823945               1.063884   
6184                               -1.657263              -1.776656   

      diff_plus_pitching_wpa_def_batted_mean  
0                                   2.137409  
1                                        NaN  
2                                        NaN  
3                                  -1.945597  


In [10]:

unique_years = test_df['year'].unique()

for yr in unique_years:
    if yr not in models:
        print(f"警告：{yr}年的模型不存在，該年份資料將跳過預測。")
        continue

    df_year = test_df[test_df['year'] == yr].copy()
 
    df_year.drop(columns=['year', 'date'], errors='ignore', inplace=True)
    
    y_pred = models[yr].predict(df_year)
    
    # 將預測結果寫回 test_df 對應的列
    test_df.loc[df_year.index, 'home_team_win'] = y_pred


submission = test_df[['id', 'home_team_win']].copy()
submission.to_csv("submission_3_1.6.csv", index=False)
print("submission.csv 已建立完成。")


submission.csv 已建立完成。


C:\Users\cheeh\AppData\Local\Temp\ipykernel_3228\1540703200.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.loc[df_year.index, 'home_team_win'] = y_pred
